In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.master("local[*]")\
        .appName("Hello")\
        .config("spark.jars.packages","org.mongodb.spark:mongo-spark-connector_2.12:2.4.1")\
        .getOrCreate()


:: loading settings :: url = jar:file:/home/fido-koffivi-gbagbe/anaconda3/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/fido-koffivi-gbagbe/.ivy2/cache
The jars for the packages stored in: /home/fido-koffivi-gbagbe/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9624439b-f65f-435f-80f5-80f272b97403;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;2.4.1 in central
	found org.mongodb#mongo-java-driver;3.10.2 in central
	[3.10.2] org.mongodb#mongo-java-driver;[3.10,3.11)
:: resolution report :: resolve 7712ms :: artifacts dl 67ms
	:: modules in use:
	org.mongodb#mongo-java-driver;3.10.2 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;2.4.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	----------------------------------------------------------

In [2]:
df= spark.read.csv("database.csv",header=True)

In [3]:
df.take(1)

[Row(Date='01/02/1965', Time='13:44:18', Latitude='19.246', Longitude='145.616', Type='Earthquake', Depth='131.6', Depth Error=None, Depth Seismic Stations=None, Magnitude='6', Magnitude Type='MW', Magnitude Error=None, Magnitude Seismic Stations=None, Azimuthal Gap=None, Horizontal Distance=None, Horizontal Error=None, Root Mean Square=None, ID='ISCGEM860706', Source='ISCGEM', Location Source='ISCGEM', Magnitude Source='ISCGEM', Status='Automatic')]

In [4]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Depth: string (nullable = true)
 |-- Depth Error: string (nullable = true)
 |-- Depth Seismic Stations: string (nullable = true)
 |-- Magnitude: string (nullable = true)
 |-- Magnitude Type: string (nullable = true)
 |-- Magnitude Error: string (nullable = true)
 |-- Magnitude Seismic Stations: string (nullable = true)
 |-- Azimuthal Gap: string (nullable = true)
 |-- Horizontal Distance: string (nullable = true)
 |-- Horizontal Error: string (nullable = true)
 |-- Root Mean Square: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Location Source: string (nullable = true)
 |-- Magnitude Source: string (nullable = true)
 |-- Status: string (nullable = true)



In [5]:
list_a_supp=["Depth Error","Depth Seismic Stations","Time","Magnitude Seismic Stations","Magnitude Error","Azimuthal",
            "Root Mean Square","Location Source","Magnitude Source","Source","Status","Azimuthal Gap","Horizontal Distance",
            "Horizontal Error"]  


In [6]:
df= df.drop(*list_a_supp)

In [7]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Depth: string (nullable = true)
 |-- Magnitude: string (nullable = true)
 |-- Magnitude Type: string (nullable = true)
 |-- ID: string (nullable = true)



In [8]:
df.show(5)

+----------+--------+---------+----------+-----+---------+--------------+------------+
|      Date|Latitude|Longitude|      Type|Depth|Magnitude|Magnitude Type|          ID|
+----------+--------+---------+----------+-----+---------+--------------+------------+
|01/02/1965|  19.246|  145.616|Earthquake|131.6|        6|            MW|ISCGEM860706|
|01/04/1965|   1.863|  127.352|Earthquake|   80|      5.8|            MW|ISCGEM860737|
|01/05/1965| -20.579| -173.972|Earthquake|   20|      6.2|            MW|ISCGEM860762|
|01/08/1965| -59.076|  -23.557|Earthquake|   15|      5.8|            MW|ISCGEM860856|
|01/09/1965|  11.938|  126.427|Earthquake|   15|      5.8|            MW|ISCGEM860890|
+----------+--------+---------+----------+-----+---------+--------------+------------+
only showing top 5 rows



In [9]:
df.dtypes

[('Date', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string'),
 ('Type', 'string'),
 ('Depth', 'string'),
 ('Magnitude', 'string'),
 ('Magnitude Type', 'string'),
 ('ID', 'string')]

In [10]:
df=df.withColumn("Year",year(to_timestamp("Date","dd/mm/yyyy")))

In [11]:
df.show(5)

+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|      Date|Latitude|Longitude|      Type|Depth|Magnitude|Magnitude Type|          ID|Year|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|01/02/1965|  19.246|  145.616|Earthquake|131.6|        6|            MW|ISCGEM860706|1965|
|01/04/1965|   1.863|  127.352|Earthquake|   80|      5.8|            MW|ISCGEM860737|1965|
|01/05/1965| -20.579| -173.972|Earthquake|   20|      6.2|            MW|ISCGEM860762|1965|
|01/08/1965| -59.076|  -23.557|Earthquake|   15|      5.8|            MW|ISCGEM860856|1965|
|01/09/1965|  11.938|  126.427|Earthquake|   15|      5.8|            MW|ISCGEM860890|1965|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
only showing top 5 rows



In [12]:
df.dtypes

[('Date', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string'),
 ('Type', 'string'),
 ('Depth', 'string'),
 ('Magnitude', 'string'),
 ('Magnitude Type', 'string'),
 ('ID', 'string'),
 ('Year', 'int')]

In [13]:
df_freq=df.groupby("Year").count().withColumnRenamed("count","Counts")
df_freq.show(5)

+----+------+
|Year|Counts|
+----+------+
|1990|   528|
|1975|   411|
|1977|   425|
|2003|   485|
|2007|   608|
+----+------+
only showing top 5 rows



In [26]:
df = df.withColumn("Latitude",df.Latitude.cast(DoubleType()))\
    .withColumn("Longitude",df.Longitude.cast(DoubleType()))\
    .withColumn("Depth",df.Depth.cast(DoubleType()))\
     .withColumn("Magnitude",df.Magnitude.cast(DoubleType()))

In [27]:
df.dtypes

[('Date', 'string'),
 ('Latitude', 'double'),
 ('Longitude', 'double'),
 ('Type', 'string'),
 ('Depth', 'double'),
 ('Magnitude', 'double'),
 ('Magnitude Type', 'string'),
 ('ID', 'string'),
 ('Year', 'int')]